In [ ]:
# ## Install Coqui TTS
# ! pip install -U pip
# ! pip install TTS==0.8.0
# import os
# !git clone https://github.com/coqui-ai/TTS
# os.chdir('TTS/')

# !python3 -m pip install -r requirements.txt
# !python3 setup.py develop

# !ls


In [ ]:
!nvidia-smi
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import torch
# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits, VitsAudioConfig
# BaseDatasetConfig: defines name, formatter and path of the dataset.


from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

from TTS.tts.configs.shared_configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig
#from TTS.configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig#GlowTTSConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager

from TTS.config import load_config

from trainer import Trainer, TrainerArgs


In [ ]:
# # !apt install -y espeak
# !apt install -y espeak-ng
!pwd

In [ ]:
!ls bn_tts

In [ ]:
import TTS
TTS.__version__

In [ ]:

# output_path = "tts_train_dir"
# if not os.path.exists(output_path):
#     os.makedirs(output_path)

male = False
pretrained = False

pretrained_path = ''
if(pretrained):
    pretrained_path = '/home/ansary/Shabab/vits_4_nov'
if(male):
    meta_file = '/home/ansary/Shabab/bn_tts/male/mono/metadata_male.txt'
    root_path = '/home/ansary/Shabab/bn_tts/male/mono'
else:
    meta_file = '/home/ansary/Shabab/bn_tts/female/mono/metadata_female.txt'
    root_path = '/home/ansary/Shabab/bn_tts/female/mono'

def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file to TTS format
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = meta_file
    items = []
    speaker_name = "ljspeech"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wav", cols[0] + ".wav")
            try:
                text = cols[1]
            except:
                print("not found")

            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


dataset_config = BaseDatasetConfig(
     meta_file_train=meta_file, path=os.path.join(root_path, "")
)


In [ ]:
dataset_config

In [ ]:
# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config,formatter=formatter, eval_split=True)

In [ ]:
print(len(train_samples),len(eval_samples))

In [ ]:
train_samples[0]

In [ ]:
# we use the same path as this script as our training folder.
output_path = '/home/ansary/Shabab/'

# audio_config = BaseAudioConfig(
#      sample_rate = 22050,
#     resample =True
# )

audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)


if(male):
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "তট৫ভিঐঋখঊড়ইজমএেঘঙসীঢ়হঞ‘ঈকণ৬ঁৗশঢঠ\u200c১্২৮দৃঔগও—ছউংবৈঝাযফ\u200dচরষঅৌৎথড়৪ধ০ুূ৩আঃপয়’নলো",
    punctuations = "-!,|.? ",
    )
else:
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "ইগং়’ুঃন১ঝূও‘ঊোছপফৈ৮ষযৎঢঈকঠিজ০৬ীটডএঅঋধচে২৩ণউয়ঢ়খলভৗসহ্ড়দথবঔাঞশরৌম—ঐআৃঘঙ\u200cঁ৪৫ত",
    punctuations = ".?-!|, ",
    )


# VitsConfig: all model related values for training, validating and testing.

config = VitsConfig(
    audio=audio_config,
    run_name="vits_4_nov",
    batch_size=104,
    eval_batch_size=48,
    batch_group_size=0,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=15000,
    text_cleaner=None, #"collapse_whitespace"
    use_phonemes=False,
    compute_input_seq_cache=True,
    print_step=50,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    characters = characters_config,
    save_step=1000,
    cudnn_benchmark=True,
    test_sentences = [
        'হয়,হয়ে,ওয়া,হয়েছ,হয়েছে,দিয়ে,যায়,দায়,নিশ্চয়,আয়,ভয়,নয়,আয়াত,নিয়ে,হয়েছে,দিয়েছ,রয়ে,রয়েছ,রয়েছে।',
        'দেয়,দেওয়া,বিষয়,হয়,হওয়া,সম্প্রদায়,সময়,হয়েছি,দিয়েছি,হয়,হয়েছিল,বিষয়ে,নয়,কিয়াম,ইয়া,দেয়া,দিয়েছে,আয়াতে,দয়া।', 
        'ইয়াহুদ,নয়,ব্যয়,ইয়াহুদী,নেওয়া,উভয়ে,যায়,হয়েছিল,প্রয়োজন।'
    ]
)


In [ ]:

ap = AudioProcessor.init_from_config(config)
# ap.resample = True

In [ ]:
ap.resample

In [ ]:

tokenizer, config = TTSTokenizer.init_from_config(config)

In [ ]:

model = Vits(config, ap, tokenizer, speaker_manager=None)

In [ ]:
# checkpoint = torch.load('/home/ansary/Shabab/vits_20_october/best_model_311129.pth')
# model.load_state_dict(checkpoint["model"])

model

In [ ]:
# use continue_path argument below for retraining model from last saved weight https://github.com/coqui-ai/Trainer/blob/main/trainer/trainer.py#L210

trainer = Trainer(
    TrainerArgs(continue_path = pretrained_path), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

In [ ]:
%%time

trainer.fit()

In [ ]:
!nvidia-smi